In [155]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split as split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
import pickle

In [67]:
bank = pd.read_csv(r"C:\Users\Administrator\Desktop\Prodigy InfoTech\Task-03\bank\bank.csv")
bank = bank.rename({"y": "subscribed"}, axis = 1)
bank = bank.drop(columns = ["job", "contact", "day", "month", "pdays", "previous", "poutcome"])
bank

,age,marital,education,default,balance,housing,loan,duration,campaign,subscribed
0,30,married,primary,no,1787,no,no,79,1,no
1,33,married,secondary,no,4789,yes,yes,220,1,no
2,35,single,tertiary,no,1350,yes,no,185,1,no
3,30,married,tertiary,no,1476,yes,yes,199,4,no
4,59,married,secondary,no,0,yes,no,226,1,no
...,...,...,...,...,...,...,...,...,...,...
4516,33,married,secondary,no,-333,yes,no,329,5,no
4517,57,married,tertiary,yes,-3313,yes,yes,153,1,no
4518,57,married,secondary,no,295,no,no,151,11,no
4519,28,married,secondary,no,1137,no,no,129,4,no


In [69]:
bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   age         4521 non-null   int64 
 1   marital     4521 non-null   object
 2   education   4521 non-null   object
 3   default     4521 non-null   object
 4   balance     4521 non-null   int64 
 5   housing     4521 non-null   object
 6   loan        4521 non-null   object
 7   duration    4521 non-null   int64 
 8   campaign    4521 non-null   int64 
 9   subscribed  4521 non-null   object
dtypes: int64(4), object(6)
memory usage: 353.3+ KB


In [107]:
marital_dummies = pd.get_dummies(bank.marital)

In [73]:
bank["education_label"] = LabelEncoder().fit_transform(bank.education)

In [83]:
bank["default_label"] = LabelEncoder().fit_transform(bank.default)
bank["housing_label"] = LabelEncoder().fit_transform(bank.housing)
bank["loan_label"] = LabelEncoder().fit_transform(bank.loan)
bank["subscribed_label"] = LabelEncoder().fit_transform(bank.subscribed)

In [117]:
df = bank.drop(columns = ["marital", "education", "default", "housing", "loan", "subscribed"])
df = pd.concat([df, marital_dummies], axis = 1).drop(columns = "divorced")

In [119]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   age               4521 non-null   int64
 1   balance           4521 non-null   int64
 2   duration          4521 non-null   int64
 3   campaign          4521 non-null   int64
 4   education_label   4521 non-null   int32
 5   default_label     4521 non-null   int32
 6   housing_label     4521 non-null   int32
 7   loan_label        4521 non-null   int32
 8   subscribed_label  4521 non-null   int32
 9   married           4521 non-null   bool 
 10  single            4521 non-null   bool 
dtypes: bool(2), int32(5), int64(4)
memory usage: 238.5 KB


In [125]:
X = df.drop(columns = "subscribed_label")
y = df.subscribed_label

In [129]:
x_train, x_test, y_train, y_test = split(X, y, test_size = 0.25, random_state = 1)

In [133]:
tree_model = DecisionTreeClassifier()
tree_model.fit(x_train, y_train)

DecisionTreeClassifier()

In [137]:
tree_model.score(x_test, y_test)

0.8099027409372237

In [139]:
y_pred = tree_model.predict(x_test)
confusion_matrix(y_pred, y_test)

array([[879, 106],
       [109,  37]], dtype=int64)

In [145]:
score_list = list(cross_val_score(tree_model, X, y, cv=5))
score_list

[0.8408839779005525,
 0.8230088495575221,
 0.8285398230088495,
 0.8617256637168141,
 0.831858407079646]

In [151]:
mean_score = np.mean(score_list)
mean_score

0.837203344252677

In [157]:
with open("Task-03", "wb") as f:
    pickle.dump(tree_model, f)